# Sequential Inference

This tutorial discusses particle filters, and sequential Monte Carlo more generally.

These techniques are relevant when performing inference on models where there is a large series of factor statements, some of which can be performed earlier than others. This situation often arises in time series models where the factor statements are the result of incoming observations, but the technique works for *any* probabilistic program.

As a motivating example, consider the following program:

In [5]:
:e ImportQualifiedPost
:e FlexibleContexts
:e BlockArguments
:e TupleSections
:e FlexibleContexts
:e OverloadedStrings
:e LambdaCase

:l Plotting.hs
:l ../src/Control/Monad/Bayes/Class.hs
:l ../src/Control/Monad/Bayes/Weighted.hs
:l ../src/Control/Monad/Bayes/Sampler.hs
:l ../src/Control/Monad/Bayes/Free.hs
:l ../src/Control/Monad/Bayes/Population.hs
:l ../src/Control/Monad/Bayes/Sequential.hs
:l ../src/Control/Monad/Bayes/Inference/SMC.hs

import qualified Graphics.Vega.VegaLite as VL
import IHaskell.Display.Hvega (vlShow)
import Control.Applicative
import qualified Data.Text as T
import Pipes (Producer, (>->))
import qualified Pipes as P
import Pipes.Prelude (unfoldr)
import qualified Pipes.Prelude as P


import Data.Ord
import Data.List
import Control.Monad
import Control.Arrow (first)

In [13]:
ex :: MonadInfer m => m Bool
ex = replicateM 100 do
    ETC

: 

What distribution does this represent? It is the distribution over all lists of Booleans of length $10$, which places all the weight on the sequence `[True,True,True,True,True,True,True,True,True,True]`.

However, the naive approach to exactly inferring this distribution will not work. Why? Because it first constructs all $2^100$ possible solutions, and then throws away all but `[True,True,True,True,True,True,True,True,True,True]`. 

Now if we look at the structure of the program, it's clear that this is unnecessary. Each time a `condition` statement is made, we should throw away all possibilities with $0$ probability mass. If we do this, the size of the set of possible solutions never explodes.

We can perform this *sequential enumeration" with monad-bayes, as follows:

In [ ]:
TODO

: 

`sis` is an inference method which performs a step of inference at each `factor` statement in the program. In the present case, we have used it in conjunction with exact inference, but the idea generalizes naturally to approximate inference methods.

To motivate this, let's examine a problem for which exact inference is no longer feasible, a non-linear state space model.



In [2]:
variance = 1

-- how to move from one latent state to the next
latentTransition :: MonadSample m => (Double, Double) -> m (Double, Double)
latentTransition (x,y) = 
    liftA2 (,) (normal (x+0.5) variance) (normal (y+0.5) variance)

walk :: MonadSample m => Producer (Double,Double) m r
walk = flip P.unfoldr (0,0) $ \s ->
    Right <$> do
        new <- latentTransition s
        return (new, new)

toList :: Monad m => P.Producer a m () -> m [a]
toList prod = P.fold  (\x y -> x <> [y]) [] id (prod >-> P.take 100)


In [3]:
observations = walk >-> P.mapM (\(x,y) -> 
    do
        x' <- normal x 2
        y' <- normal y 2
        return (x, y, x', y'))

In [4]:
(xs, ys, xs', ys') <- sampleIOfixed $ unzip4 <$> toList observations

plotVega 
    (zip (zip (xs <> xs') (ys <> ys')) 
    (replicate 100 (T.pack "Latent") <> replicate 100 (T.pack "Observed")))

In [11]:
conditioning :: (MonadSample m, MonadCond m) => P.Producer (Double,Double) m ()
conditioning = 
    P.zip walk (P.each (zip xs' ys')) 
    >-> P.chain (\((x,y), (x',y')) -> factor (normalPdf x variance x' * normalPdf y variance y' ))
    >-> P.map fst 

model :: MonadInfer m => m [(Double,Double)]
model = toList conditioning

model :: forall (m :: * -> *). (MonadSample m, MonadCond m) => m [(Double, Double)]

`model` is the distribution we want to perform inference on.

The intuition of the problem is that we need to guess the whole path, by taking into account not just the observation at each timestep, but the relation (expressed in the prior) of the inferred position at that timestep to the previous and subsequent ones.

To perform approximate inference here, we can represent a distribution as a *population* of weighted samples, and at each factor statement, do a resampling operation to keep the population "healthy". This is **Sequential Monte Carlo**. 

<!-- The algorithm creates a population of $n$ samples (initially weighted equally) and updates them at each factor statement. The update is defined by `smcMultinomial`, which samples new samples from the empirical distribution defined by the population. More sophisticated resamplers are available, including `smcSystematic` (see TODO) -->

In [20]:
smcRes <- sampleIOfixed $ runPopulation $ smcMultinomial 100 20 model

In [21]:
(inferredXs, inferredYs) = unzip $ fst =<< smcRes

plotVega 
    (zip (zip (xs <> inferredXs) (ys <> inferredYs)) 
    (replicate 100 (T.pack "Latent") <> replicate 100 (T.pack "Inferred")))

We can see here that `SMC` has done a pretty good job.

The intuition of how SMC does well is that it is searching the space of solutions with some backtracking (because it has a whole population of guesses to rely on), and moreover, this search results in unbiased sampling from the distribution of interest. So if it makes a series of guesses the leads it astray, it can throw away this possiblity at a later stage. 